# Etapa 1 (Landing & Bronze)
Aquisição de Dados e Armazenamento de Dados em PostgreSQL.
- Baixe o conjunto de dados "Inside Airbnb" do Rio de Janeiro da fonte oficial (http://insideairbnb.com/) e promova uma estruturação simples nos dados.
- Crie um banco de dados PostgreSQL para armazenar os dados brutos das 3 tabelas ("Listing", "Reviews" e Calendar") na camada "bronze".

## Landing

In [3]:
from modules.etl import Landing

url = [
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/listings.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/calendar.csv.gz',
    'https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/reviews.csv.gz'
]
landing_step = Landing(landing_path='data/landing')
landing_step.extract(url=url)
landing_step.transform()

Efetuando a requisição para a url https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/listings.csv.gz
Efetuando a requisição para a url https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/calendar.csv.gz
Efetuando a requisição para a url https://data.insideairbnb.com/brazil/rj/rio-de-janeiro/2023-12-26/data/reviews.csv.gz
Salvando o arquivo data/landing/reviews.csv.gz
Salvando o arquivo data/landing/calendar.csv.gz
Salvando o arquivo data/landing/listings.csv.gz


## Bronze

In [2]:
import os
import pandas as pd
from modules.utils import clean_path
from modules.database import PostgreSQL

## Criação de base de dados
database = PostgreSQL() # Inicializando a classe com as variaveis de conexão com o banco
database.create_database('datalake') # Criando o database datalake

## Criação dos Schema
try:
    print("Iniciando a criação dos schemas bronze, silver e gold")

    database.create_schema(schema_name='bronze',database='datalake')
    database.create_schema(schema_name='silver',database='datalake')
    database.create_schema(schema_name='gold',database='datalake')

    print("Fim da criação dos schemas")
except Exception as e:
    print(f"Erro ao criar os schemas: {e}")
    raise e

## Inserção dos dataframes
try:
    print("Inicio do carregamento de dados (Bronze)")

    data_path = './data/' # Define onde estão os CSVs
    df_listings = pd.read_csv(os.path.join(data_path,'listings.csv')) # Lê o csv e transforma em um dataframe Python
    database.insert_dataframe(dataframe=df_listings,table_name='listings',schema='bronze',database='datalake',if_exists='append') # Insere o Dataframe no banco de dados
    del df_listings # Libera o espaço que a variável consome na memória RAM

    df_reviews = pd.read_csv(os.path.join(data_path,'reviews.csv')) # Lê o csv e transforma em um dataframe Python
    database.insert_dataframe(dataframe=df_reviews,table_name='reviews',schema='bronze',database='datalake',if_exists='append') # Insere o Dataframe no banco de dados
    del df_reviews # Libera o espaço que a variável consome na memória RAM

    df_calendar = pd.read_csv(os.path.join(data_path,'calendar.csv')) # Lê o csv e transforma em um dataframe Python
    database.insert_dataframe(dataframe=df_calendar,table_name='calendar',schema='bronze',database='datalake',if_exists='append') # Insere o Dataframe no banco de dados
    del df_calendar # Libera o espaço que a variável consome na memória RAM

    clean_path(path='data/landing') # Limpando os arquivos CSVs

    print("Fim do carregamento de dados (Bronze)")
except Exception as e:
    print(f"Erro ao inserir os dados: {e}")
    raise e

ProgrammingError: (psycopg2.errors.SyntaxError) option "locale_provider" not recognized
LINE 6:                 LOCALE_PROVIDER = 'libc'
                        ^

[SQL: 
            CREATE DATABASE datalake
                WITH
                OWNER = postgres
                ENCODING = 'UTF8'
                LOCALE_PROVIDER = 'libc'
                CONNECTION LIMIT = -1
                IS_TEMPLATE = False;
            ]
(Background on this error at: https://sqlalche.me/e/20/f405)